In [128]:
import pandas as pd
import requests
from datetime import datetime,timedelta

import warnings
warnings.filterwarnings('ignore')

In [129]:
import numpy as np

In [130]:
col_name=["TM", "STN", "WD", "WS", "GST_WD", "GST_WS", "GST_TM", "PA", "PS", "PT", "PR", 
          "TA", "TD", "HM", "PV", "RN", "RN_DAY", "RN_JUN", "RN_INT", "SD_HR3", "SD_DAY",
          "SD_TOT", "WC", "WP", "WW", "CA_TOT", "CA_MID", "CH_MIN", "CT", "CT_TOP", "CT_MID",
          "CT_LOW", "VS", "SS", "SI", "ST_GD", "TS", "TE_005", "TE_01", "TE_02", "TE_03", "ST_SEA",
              "WH", "BF", "IR", "IX" ]

def weather(tm):
    url="https://apihub.kma.go.kr/api/typ01/url/kma_sfctm2.php"
    key="GGGECxLiR_qhhAsS4sf6xA"
    params={'tm':tm,
            'stn':152,
            'help':0,
            'authKey':key}
    response=requests.get(url, params=params, verify=False)
    text=response.text
    text=text.split("\n")[4:-2][0]
    text = text.split()

    df=pd.DataFrame([text], columns=col_name)
    # # df=df[0].str.split(expand=True)
    # print((df.columns))
    # df.columns=col_name
    return df



In [131]:
df=weather("202506170000")
df["TA"]

0    18.9
Name: TA, dtype: object

In [132]:
df

,TM,STN,WD,WS,GST_WD,GST_WS,GST_TM,PA,PS,PT,...,TS,TE_005,TE_01,TE_02,TE_03,ST_SEA,WH,BF,IR,IX
0,202506170000,152,7,0.8,-9,-9.0,-9,994.3,1003.6,2,...,20.0,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,1


In [133]:
import requests
import pandas as pd
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)  # verify=False 경고 제거

# ▶ 사용자 인증키 입력
auth_key = 'GGGECxLiR_qhhAsS4sf6xA'

# ▶ 요청 URL 및 파라미터 설정
url = "https://apihub.kma.go.kr/api/typ01/url/kma_sfctm2.php"
params = {
    'inf': 'SFC',       # 초단기 관측
    'stn': '109',          # 전체 지역
    'authKey': 'GGGECxLiR_qhhAsS4sf6xA'
}

# ▶ 요청 및 응답 파싱
response = requests.get(url, params=params, verify=False)
text = response.text.replace("#", "").split('\n')

# ▶ 빈 줄 제거 및 공백 -> , 로 변환, 연속된 , 제거
parsed = []
for line in text:
    line = line.strip().replace(" ", ",").split(',')
    line = [item for item in line if item != '']
    if line:
        parsed.append(line)

# ▶ 열 이름 처리 (3번째 줄부터 시작)
col_name1 = parsed[2]
col_name2 = [c.replace("-", "") for c in parsed[3]]
col_name = [f"{n1}({n2})" for n1, n2 in zip(col_name1, col_name2)]
col_name = [c.replace("()", "") for c in col_name]

# ▶ 데이터 행 추출
data_rows = parsed[4:]

# ▶ 데이터프레임 생성 (열 수 안 맞는 행 자동 보정)
df = pd.DataFrame(data_rows)
df = df.loc[:, :len(col_name)-1]  # 열 수 자르기
df.columns = col_name[:df.shape[1]]  # 컬럼명 지정




In [134]:
df

,YYMMDDHHMI(KST)
0,7777END


In [135]:
seoul_gu_coords = {
    "종로구": (37.572950, 126.979357),
    "중구": (37.563757, 126.997128),
    "용산구": (37.532600, 126.990600),
    "성동구": (37.563619, 127.036260),
    "광진구": (37.538123, 127.082190),
    "동대문구": (37.574360, 127.039320),
    "중랑구": (37.606611, 127.092950),
    "성북구": (37.589000, 127.016900),
    "강북구": (37.639361, 127.025341),
    "도봉구": (37.668759, 127.047439),
    "노원구": (37.654278, 127.056732),
    "은평구": (37.602636, 126.929054),
    "서대문구": (37.579472, 126.936841),
    "마포구": (37.566631, 126.901546),
    "양천구": (37.517517, 126.866323),
    "강서구": (37.550965, 126.849533),
    "구로구": (37.495757, 126.887662),
    "금천구": (37.456255, 126.895253),
    "영등포구": (37.526617, 126.896800),
    "동작구": (37.502775, 126.940545),
    "관악구": (37.478644, 126.951731),
    "서초구": (37.483500, 127.032600),
    "강남구": (37.517236, 127.047324),
    "송파구": (37.514589, 127.105386),
    "강동구": (37.530125, 127.123424)
}


In [136]:
import math

def latlon_to_grid(lat, lon):
    RE = 6371.00877  # 지구 반경(km)
    GRID = 5.0       # 격자 간격(km)
    SLAT1 = 30.0     # 투영 위도1(degree)
    SLAT2 = 60.0     # 투영 위도2(degree)
    OLON = 126.0     # 기준점 경도(degree)
    OLAT = 38.0      # 기준점 위도(degree)
    XO = 43          # 기준점 X좌표(GRID)
    YO = 136         # 기준점 Y좌표(GRID)

    DEGRAD = math.pi / 180.0

    re = RE / GRID
    slat1 = SLAT1 * DEGRAD
    slat2 = SLAT2 * DEGRAD
    olon = OLON * DEGRAD
    olat = OLAT * DEGRAD

    sn = math.tan(math.pi * 0.25 + slat2 * 0.5) / math.tan(math.pi * 0.25 + slat1 * 0.5)
    sn = math.log(math.cos(slat1) / math.cos(slat2)) / math.log(sn)
    sf = math.tan(math.pi * 0.25 + slat1 * 0.5)
    sf = (sf ** sn * math.cos(slat1)) / sn
    ro = math.tan(math.pi * 0.25 + olat * 0.5)
    ro = re * sf / (ro ** sn)
    ra = math.tan(math.pi * 0.25 + lat * DEGRAD * 0.5)
    ra = re * sf / (ra ** sn)
    theta = lon * DEGRAD - olon
    if theta > math.pi:
        theta -= 2.0 * math.pi
    if theta < -math.pi:
        theta += 2.0 * math.pi
    theta *= sn
    x = (ra * math.sin(theta)) + XO + 0.5
    y = (ro - ra * math.cos(theta)) + YO + 0.5
    return int(x), int(y)


In [145]:
import requests
import pandas as pd
from datetime import datetime, timedelta
def weather(region):
    # 1. 격자 좌표 가져오기
    file_path = '기상청.xlsx'
    df = pd.read_excel(file_path, sheet_name='최종 업데이트 파일_20241031')
    # 예: 서울특별시 종로구
    target = df[(df['1단계'] == '서울특별시') & (df['2단계'] == region)].iloc[0]
    nx = int(target['격자 X'])  # X좌표
    ny = int(target['격자 Y'])  # Y좌표

    # 2. 기준 날짜와 시간 계산 (예: 현재 시각 기준으로 가장 최근 발표 시간)
    now = datetime.now()
    base_time = now.replace(minute=0, second=0, microsecond=0)

    # 1시간 간격으로 발표됨 → 40분 이전이면 한 시간 전 데이터 요청
    if now.minute < 45:
        base_time -= timedelta(hours=1)

    base_date = base_time.strftime("%Y%m%d")
    base_time = base_time.strftime("%H%M")

    # 3. API 요청 URL 구성
    url = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst"

    params = {
        'serviceKey': 'vlFqNOMGHQuVJ6GoaP8C99d89CrkizjL/eMTJtgiLYQVr9Sbmo0CKfDHKR/6WhBptBwMDguCzQQxFMZHhKxSCw==',  # Decoding 된 인증키
        'pageNo': '1',
        'numOfRows': '1000',
        'dataType': 'JSON',
        'base_date': base_date,
        'base_time': base_time,
        'nx': nx,
        'ny': ny
    }

    # 4. API 호출
    response = requests.get(url, params=params)
    data = response.json()

    # 5. 결과 확인
    if 'response' in data and data['response']['header']['resultCode'] == '00':
        items = data['response']['body']['items']['item']
        df_result = pd.DataFrame(items)
        print(df_result[['category', 'fcstTime', 'fcstValue']])
        return df_result
    else:
        print("API 호출 실패:", data)

    


In [146]:
weather('마포구')

   category fcstTime fcstValue
0       LGT     1700         0
1       LGT     1800         0
2       LGT     1900         0
3       LGT     2000         0
4       LGT     2100         0
5       LGT     2200         0
6       PTY     1700         0
7       PTY     1800         0
8       PTY     1900         0
9       PTY     2000         0
10      PTY     2100         0
11      PTY     2200         0
12      RN1     1700      강수없음
13      RN1     1800      강수없음
14      RN1     1900      강수없음
15      RN1     2000      강수없음
16      RN1     2100      강수없음
17      RN1     2200      강수없음
18      SKY     1700         1
19      SKY     1800         1
20      SKY     1900         1
21      SKY     2000         1
22      SKY     2100         1
23      SKY     2200         1
24      T1H     1700        26
25      T1H     1800        25
26      T1H     1900        25
27      T1H     2000        23
28      T1H     2100        22
29      T1H     2200        21
30      REH     1700        60
31      

,baseDate,baseTime,category,fcstDate,fcstTime,fcstValue,nx,ny
0,20250617,1630,LGT,20250617,1700,0,59,127
1,20250617,1630,LGT,20250617,1800,0,59,127
2,20250617,1630,LGT,20250617,1900,0,59,127
3,20250617,1630,LGT,20250617,2000,0,59,127
4,20250617,1630,LGT,20250617,2100,0,59,127
5,20250617,1630,LGT,20250617,2200,0,59,127
6,20250617,1630,PTY,20250617,1700,0,59,127
7,20250617,1630,PTY,20250617,1800,0,59,127
8,20250617,1630,PTY,20250617,1900,0,59,127
9,20250617,1630,PTY,20250617,2000,0,59,127
